In [9]:
%load_ext autoreload
%autoreload 2

In [10]:
import numpy as np
from pyscf import fci, gto, scf

np.set_printoptions(precision=5, suppress=True)

In [39]:
r = 1.6
nH = 10
# geom = f"H {-3*r/2} 0 0; H {-r/2} 0 0; H {r/2} 0 0; H {3*r/2} 0 0"
geom = ""
for i in range(nH):
    geom += "H 0 0 %g\n" % (i * r)
mol = gto.M(atom=geom, basis="sto-6g", verbose=3, unit="bohr")

mf = scf.RHF(mol)
mf.kernel()

umf = scf.UHF(mol)
umf.kernel()
mo1 = umf.stability(external=True)[0]
umf = umf.newton().run(mo1, umf.mo_occ)
mo1 = umf.stability(external=True)[0]
umf = umf.newton().run(mo1, umf.mo_occ)

# fci
cisolver = fci.FCI(mol, mf.mo_coeff)
e, ci = cisolver.kernel()
print("FCI energy: ", e)

converged SCF energy = -5.25628158758201
converged SCF energy = -5.25628158758181  <S^2> = 2.5845992e-13  2S+1 = 1
<class 'pyscf.scf.uhf.UHF'> wavefunction is stable in the internal stability analysis
<class 'pyscf.scf.uhf.UHF'> wavefunction is stable in the real -> complex stability analysis
<class 'pyscf.scf.uhf.UHF'> wavefunction is stable in the UHF/UKS -> GHF/GKS stability analysis
converged SCF energy = -5.25628158758199  <S^2> = 6.2172489e-14  2S+1 = 1
<class 'pyscf.soscf.newton_ah.newton.<locals>.SecondOrderUHF'> wavefunction is stable in the internal stability analysis
<class 'pyscf.soscf.newton_ah.newton.<locals>.SecondOrderUHF'> wavefunction is stable in the real -> complex stability analysis
<class 'pyscf.soscf.newton_ah.newton.<locals>.SecondOrderUHF'> wavefunction is stable in the UHF/UKS -> GHF/GKS stability analysis
converged SCF energy = -5.25628158758202  <S^2> = 3.0198066e-14  2S+1 = 1
FCI energy:  -5.3843610660956


### Free projection AFQMC

Evaluates the quantity

$$ E(\tau) = \frac{\langle \Psi_l | H e^{-\tau H} | \Psi_r \rangle}{\langle \Psi_l | e^{-\tau H} | \Psi_r \rangle} $$

where $|\Psi_l\rangle$ is a trial wave function and $|\Psi_r\rangle$ is an initial state. The propagator is sampled using Monte Carlo. $E(\tau)$ converges to the ground state energy at long $\tau$, but the energies get noisier at long $\tau$ due to the sign problem.

In the following, energy evaluations are performed after a block consisting of `num_steps` steps of duration `dt`. In one iteration, energy samples are collected at `num_blocks` different $\tau$ values. Multiple walkers are used to batch operations together for computational efficiency. The total number of samples at a given $\tau$ is given by `num_walkers` $\times$ `num_iterations_fp`. The energy is then averaged over walkers and iterations.


In [40]:
from ipie.qmc.calc import build_fpafqmc_driver
from ipie.config import MPI
from ipie.utils.from_pyscf import gen_ipie_input_from_pyscf_chk
from ipie.analysis.blocking import jackknife_ratios

comm = MPI.COMM_WORLD

gen_ipie_input_from_pyscf_chk(umf.chkfile, verbose=0)
qmc_options = {
    "num_iterations_fp": 100,
    "num_blocks": 4,
    "num_steps": 30,
    "num_walkers": 50,
    "dt": 0.05,
}
afqmc = build_fpafqmc_driver(
    comm,
    nelec=mol.nelec,
    seed=212503,
    qmc_options=qmc_options,
)
afqmc.run()

# analysis
from ipie.analysis.extraction import extract_observable_complex

for i in range(afqmc.params.num_blocks):
    print(
        f"\nEnergy statistics at time {(i+1) * afqmc.params.num_steps_per_block * afqmc.params.timestep}:"
    )
    qmc_data = extract_observable_complex(afqmc.estimators[i].filename, "energy")
    jackknife_ratios(qmc_data["ENumer"], qmc_data["EDenom"])

# Note: batched not specified. Setting to default value of True.
# MPIHandler detected 1 groups with 1 members each
# Parsing input options for systems.Generic.
# Number of alpha electrons: 5
# Number of beta electrons: 5
# Setting integrals to hamiltonian.h5.
# Note: symmetry not specified. Setting to default value of True.
# Have shared memory: True
# Time to read integrals: 0.001281
# Time to pack Cholesky vectors: 0.000158
# Number of orbitals: 10
# Approximate memory required by Cholesky vectors 0.000020 GB
# Approximate memory required by packed Cholesky vectors 0.000011 GB
# Approximate memory required total 0.000031 GB
# Number of Cholesky vectors: 27
# Number of fields: 27
# Finished setting up GenericRealChol object.
# random seed is 212503
# ipie version: 0.7.1.dev0
# Git hash: e4163010b903af9fcab8f8f4180e4b1d18c7a46d-dirty.
# Git branch: develop.
# Found uncommitted changes and/or untracked files.
# Modified : ipie/analysis/blocking.py
# Modified : ipie/analysis/extraction.

### Phaseless AFQMC

In [41]:
from ipie.qmc.calc import build_afqmc_driver
from ipie.config import MPI
from ipie.utils.from_pyscf import gen_ipie_input_from_pyscf_chk

comm = MPI.COMM_WORLD

gen_ipie_input_from_pyscf_chk(mf.chkfile, verbose=0)

# fixing random seed for reproducibility
afqmc = build_afqmc_driver(comm, nelec=mol.nelec, num_walkers_per_task=100, seed=41100801)
if comm.rank == 0:
    print(afqmc.params)  # Inspect the default qmc options

# Let us override the number of blocks to keep it short
afqmc.params.num_blocks = 400
afqmc.run()

if comm.rank == 0:
    # We can extract the qmc data as as a pandas data frame like so
    from ipie.analysis.extraction import extract_observable

    qmc_data = extract_observable(afqmc.estimators.filename, "energy")
    y = qmc_data["ETotal"]
    y = y[50:]  # discard first 50 blocks

    from ipie.analysis.autocorr import reblock_by_autocorr

    df = reblock_by_autocorr(y, verbose=1)
    print(df.to_csv(index=False))
    # assert np.isclose(df.at[0,'ETotal_ac'], -5.325611614468466)
    # assert np.isclose(df.at[0,'ETotal_error_ac'], 0.00938082351500978)

# Note: batched not specified. Setting to default value of True.
# MPIHandler detected 1 groups with 1 members each
# Parsing input options for systems.Generic.
# Number of alpha electrons: 5
# Number of beta electrons: 5
# Setting integrals to hamiltonian.h5.
# Note: symmetry not specified. Setting to default value of True.
# Have shared memory: True
# Time to read integrals: 0.001227
# Time to pack Cholesky vectors: 0.000163
# Number of orbitals: 10
# Approximate memory required by Cholesky vectors 0.000020 GB
# Approximate memory required by packed Cholesky vectors 0.000011 GB
# Approximate memory required total 0.000031 GB
# Number of Cholesky vectors: 27
# Number of fields: 27
# Finished setting up GenericRealChol object.
# random seed is 41100801
QMCParams(num_walkers=100, total_num_walkers=100, timestep=0.005, num_steps_per_block=25, num_blocks=1000, num_stblz=5, pop_control_freq=5, rng_seed=41100801)
# Using pair_branch population control algorithm.
# target weight is 100
# tot

/Users/ankitmahajan/softwares/ipie/ipie/analysis/autocorr.py:92: FutureWarning: The behavior of `series[i:j]` with an integer-dtype index is deprecated. In a future version, this will be treated as *label-based* indexing, consistent with e.g. `series[i]` lookups. To retain the old behavior, use `series.iloc[i:j]`. To get the future behavior, use `series.loc[i:j]`.
  tacs += [autocorr_gw2010(y[:n])]
/Users/ankitmahajan/softwares/ipie/ipie/analysis/autocorr.py:107: FutureWarning: The behavior of `series[i:j]` with an integer-dtype index is deprecated. In a future version, this will be treated as *label-based* indexing, consistent with e.g. `series[i]` lookups. To retain the old behavior, use `series.iloc[i:j]`. To get the future behavior, use `series.loc[i:j]`.
  yblocked += [numpy.mean(y[offset : offset + block_size])]
/Users/ankitmahajan/softwares/ipie/ipie/analysis/autocorr.py:105: FutureWarning: The behavior of `series[i:j]` with an integer-dtype index is deprecated. In a future vers